# Data Pull

In [1]:
pip install ccxt

In [2]:
import ccxt
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone
from tqdm import tqdm
import os

## Coin Universe

In [3]:
SPOT_COLUMNS = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume',
                'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

binance_exchange = ccxt.binanceus({
    'timeout': 15000,
    'enableRateLimit': True
    # 'options': {'defaultType': 'future'}
})
markets = binance_exchange.load_markets()
all_tickers = list(markets.keys())
usdt_tickers = [ticker for ticker in all_tickers if ticker.endswith('USDT')]
active_usdt_tickers = [
    ticker for ticker in usdt_tickers if markets[ticker]['active']
]
len(active_usdt_tickers)

162

In [4]:
def get_spot(exchange, symbol, interval = '1d',
                    startTime = None,
                    endTime = None,
                    limit = 1000):

    if (startTime == None and endTime == None):
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'limit': limit})
    elif (startTime == None and endTime != None):
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'endTime': endTime,
                                        'limit': limit})
    elif (startTime != None and endTime == None):
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'startTime': startTime,
                                        'limit': limit})
    else:
        return exchange.publicGetKlines({'symbol': symbol,
                                        'interval': interval,
                                        'startTime': startTime,
                                        'endTime': endTime,
                                        'limit': limit})

def convert_to_seconds(time_input):
    number = int(time_input[:-1])
    unit = time_input[-1]

    if unit == 's':
        return number
    elif unit == 'm':
        return number * 60
    elif unit == 'h':
        return number * 3600
    elif unit == 'd':
        return number * 86400
    else:
        raise ValueError("Unsupported time unit")

def transform_timestamp(timestamp_integer):
    '''
    As data points involved milliseconds, we need to transform them by constant 1000.
    '''

    return pd.to_datetime(int(timestamp_integer / 1000), utc=True, unit='s')

def transform_to_timestamp_integer(datetime_object):
    '''
    As data points involved milliseconds, we need to transform them by constant 1000.
    '''

    return int(datetime_object.timestamp() * 1000)

def obtain_full_spotdata(start_timestamp, end_timestamp, exchange, symbol, interval='1d', limit=1000):
    full_data_list = []
    curr_timestamp = start_timestamp

    while curr_timestamp < end_timestamp:
        data_list = get_spot(exchange=exchange, symbol=symbol, interval=interval, startTime=curr_timestamp, limit=limit)
        if not data_list:
            break

        full_data_list.extend(data_list)

        # Use the last candle's open time plus one interval as the new start timestamp
        last_candle = data_list[-1]
        last_timestamp = int(last_candle[0])
        # Increment by one day (in milliseconds) to avoid overlap
        curr_timestamp = last_timestamp + convert_to_seconds(interval) * 1000

        # Sleep to respect rate limits
        time.sleep(0.2)

    return full_data_list

In [5]:
start_date = "2016-01-01"
end_date = "2025-02-24"
interval = "1d"

In [6]:
def fetch_data(ticker_symbol, start_date, end_date, interval):
    # Convert dates to timestamps in milliseconds
    start_timestamp = int(((pd.to_datetime(start_date)).tz_localize('UTC')).timestamp() * 1000)
    end_timestamp = int(((pd.to_datetime(end_date)).tz_localize('UTC')).timestamp() * 1000)

    # Obtain raw data from Binance US
    full_data_list = obtain_full_spotdata(start_timestamp, end_timestamp, binance_exchange, ticker_symbol, interval=interval)

    # Create a DataFrame with the correct column names
    data = pd.DataFrame(full_data_list, columns=SPOT_COLUMNS)
    data['Open time'] = data['Open time'].apply(lambda x: transform_timestamp(int(x)))

    # Remove any duplicate entries based on 'Open time'
    data.drop_duplicates('Open time', keep='first', inplace=True)

    # Add a column to identify the ticker
    data['Ticker'] = ticker_symbol
    return data

In [7]:
all_data = []
for ticker in tqdm(active_usdt_tickers, desc="Fetching crypto data"):
    try:
        cleaned_ticker = ticker.replace('/', '')
        df = fetch_data(cleaned_ticker, start_date, end_date, interval)
        all_data.append(df)
        print(f"Data for {cleaned_ticker} fetched successfully.")
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Concatenate all the DataFrames into one master DataFrame
master_data = pd.concat(all_data, ignore_index=True)

# Save the master DataFrame to a Parquet file
master_data.to_parquet("master_crypto_data.parquet", index=False)
print("Master parquet file saved as 'master_crypto_data.parquet'")

Fetching crypto data:   1%|          | 1/162 [00:01<03:33,  1.32s/it]

Data for BTCUSDT fetched successfully.


Fetching crypto data:   1%|          | 2/162 [00:03<04:17,  1.61s/it]

Data for ETHUSDT fetched successfully.


Fetching crypto data:   2%|▏         | 3/162 [00:04<04:19,  1.64s/it]

...

Fetching crypto data:  99%|█████████▉| 161/162 [01:39<00:00,  3.22it/s]

Data for TRUMPUSDT fetched successfully.


Fetching crypto data: 100%|██████████| 162/162 [01:40<00:00,  1.62it/s]

Data for NEIROUSDT fetched successfully.


Master parquet file saved as 'master_crypto_data.parquet'


## Data Cleaning

In [8]:
# prompt: Only keep the Y-M-D of Open Time

import pandas as pd

data = pd.read_parquet("master_crypto_data.parquet")

data['Open Time'] = pd.to_datetime(data['Open time'])
data['Open Time (Y-M-D)'] = data['Open time'].dt.strftime('%Y-%m-%d')
data.set_index('Open Time (Y-M-D)', inplace=True)
data


,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore,Ticker,Open Time
Open Time (Y-M-D),,,,,,,,,,,,,,
2019-09-23,2019-09-23 00:00:00+00:00,9930.13000000,9930.13000000,9930.13000000,9930.13000000,0.00100000,1569283199999,9.93013000,1,0.00000000,0.00000000,0,BTCUSDT,2019-09-23 00:00:00+00:00
2019-09-24,2019-09-24 00:00:00+00:00,9637.93000000,9665.05000000,7996.45000000,8519.67000000,202.35079900,1569369599999,1739432.08920509,2345,125.93546400,1075623.60280331,0,BTCUSDT,2019-09-24 00:00:00+00:00
2019-09-25,2019-09-25 00:00:00+00:00,8528.12000000,8724.77000000,8247.78000000,8435.28000000,55.41797100,1569455999999,469361.69055629,763,29.05942700,246049.20514584,0,BTCUSDT,2019-09-25 00:00:00+00:00
2019-09-26,2019-09-26 00:00:00+00:00,8434.45000000,8453.20000000,7761.87000000,8064.41000000,41.88007900,1569542399999,340923.66601370,1023,19.80884900,161115.44149782,0,BTCUSDT,2019-09-26 00:00:00+00:00
2019-09-27,2019-09-27 00:00:00+00:00,8078.75000000,8259.96000000,7858.58000000,8178.73000000,47.14290400,1569628799999,379917.99846542,990,23.90374200,192755.96772991,0,BTCUSDT,2019-09-27 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-22,2025-02-22 00:00:00+00:00,0.00036830,0.00044590,0.00035281,0.00043700,7017510.00,1740268799999,2906.92565489,58,6191501.00,2586.24623479,0,NEIROUSDT,2025-02-22 00:00:00+00:00
2025-02-23,2025-02-23 00:00:00+00:00,0.00039728,0.00043262,0.00037002,0.00038312,14115426.00,1740355199999,5717.43299936,33,10311821.00,4228.42642474,0,NEIROUSDT,2025-02-23 00:00:00+00:00
2025-02-24,2025-02-24 00:00:00+00:00,0.00038312,0.00042142,0.00032220,0.00032220,211463527.00,1740441599999,68355.16999234,84,74092.00,29.99931429,0,NEIROUSDT,2025-02-24 00:00:00+00:00


In [11]:
pivot = data.pivot(columns='Ticker', values='Close')
pivot

Ticker,1INCHUSDT,AAVEUSDT,ACHUSDT,ADAUSDT,ADXUSDT,ALGOUSDT,ALICEUSDT,ALPINEUSDT,ANKRUSDT,APEUSDT,...,XECUSDT,XLMUSDT,XNOUSDT,XRPUSDT,XTZUSDT,YFIUSDT,ZECUSDT,ZENUSDT,ZILUSDT,ZRXUSDT
Open Time (Y-M-D),,,,,,,,,,,,,,,,,,,,,
2019-09-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.27311000,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.23264000,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-25,NaN,NaN,NaN,0.03900000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.05727000,NaN,0.24603000,NaN,NaN,NaN,NaN,NaN,0.21020000
2019-09-26,NaN,NaN,NaN,0.03789000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.05861000,NaN,0.24301000,NaN,NaN,NaN,NaN,NaN,0.21820000
2019-09-27,NaN,NaN,NaN,0.03914000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.05729000,NaN,0.24092000,NaN,NaN,NaN,NaN,NaN,0.21320000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-22,0.27150000,248.39000000,0.03336000,0.77380000,0.13920000,0.26610000,0.69100000,1.12500000,0.02422000,0.73000000,...,0.00002634,0.33440000,1.25900000,2.57580000,0.88400000,5909.00000000,36.53000000,13.62000000,0.01462000,0.32030000
2025-02-23,0.26040000,246.44000000,0.03150000,0.77130000,0.14080000,0.25950000,0.77300000,1.12500000,0.02376000,0.70100000,...,0.00002643,0.33390000,1.19200000,2.57040000,0.87300000,6299.00000000,35.75000000,13.48000000,0.01469000,0.31500000
2025-02-24,0.24070000,210.08000000,0.02830000,0.68120000,0.14080000,0.22860000,0.64100000,1.12500000,0.02060000,0.58700000,...,0.00002382,0.29850000,1.05700000,2.27430000,0.76700000,5500.00000000,33.23000000,11.76000000,0.01310000,0.28640000


In [12]:
for index in pivot.index:
    if not pivot.loc[index].isnull().any():
        print(f"The first index with no NaN values is: {index}")
        break
else:
    print("No index found without NaN values.")


The first index with no NaN values is: 2025-02-13
